In [1]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# download testing data from open datasets
testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size = 64

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for x, y in test_dataloader:
  print(x.shape, y.shape)
  break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# define model

class Fashion_model(nn.Module):
  def __init__(self):
    super(Fashion_model, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [18]:
model = Fashion_model().to(device)
print(model)

Fashion_model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.001)

In [20]:
# train function

def train(dataloader, model, loss_fn, optimiser):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # backpropagation
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if batch % 100 == 0:
      loss_, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  number_of_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  test_loss /= number_of_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [21]:
epochs = 10
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train(train_dataloader, model, loss_fn, optimiser)
  test(test_dataloader, model, loss_fn)

Epoch: 1
loss: 2.311793  [    0/60000]
loss: 2.298703  [ 6400/60000]
loss: 2.279654  [12800/60000]
loss: 2.265211  [19200/60000]
loss: 2.251946  [25600/60000]
loss: 2.218106  [32000/60000]
loss: 2.232249  [38400/60000]
loss: 2.207912  [44800/60000]
loss: 2.195555  [51200/60000]
loss: 2.156556  [57600/60000]
Test Error: 
 Accuracy: 27.8%, Avg loss: 2.159263 

Epoch: 2
loss: 2.174975  [    0/60000]
loss: 2.163758  [ 6400/60000]
loss: 2.115661  [12800/60000]
loss: 2.120955  [19200/60000]
loss: 2.066720  [25600/60000]
loss: 2.013967  [32000/60000]
loss: 2.036472  [38400/60000]
loss: 1.974172  [44800/60000]
loss: 1.972194  [51200/60000]
loss: 1.895564  [57600/60000]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.900099 

Epoch: 3
loss: 1.937354  [    0/60000]
loss: 1.901482  [ 6400/60000]
loss: 1.803087  [12800/60000]
loss: 1.826046  [19200/60000]
loss: 1.709236  [25600/60000]
loss: 1.677486  [32000/60000]
loss: 1.683747  [38400/60000]
loss: 1.605413  [44800/60000]
loss: 1.620637  [51200/60000]

In [22]:
# save model
torch.save(model.state_dict(), "model.pth")


In [23]:
# prediction

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = testing_data[1][0], testing_data[1][1]
x = x.to(device)
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f"Predicted:{predicted}, Actual:{actual}")

Predicted:Pullover, Actual:Pullover
